<a href="https://colab.research.google.com/github/SalarNouri/Advanced-Deep-Learning/blob/main/WIT/models_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#@title Import nesecassary libraries

import pandas as pd
import numpy as np
import tensorflow as tf

from keras import models
from tensorflow_addons import metrics
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder


In [ ]:
#@title Load dataset

from google.colab import files
uploaded = files.upload()

In [23]:
!tar -xvf  '/content/isfahan_models1.tar.gz'

build/
build/assets/
build/variables/
build/variables/variables.data-00000-of-00001
build/variables/variables.index
build/saved_model.pb


In [15]:
#@title Install the What-If Tool widget if running in colab {display-mode: "form"}

# If running in colab then pip install, otherwise no need.
try:
  import google.colab
  !pip install --upgrade tensorflow>=2.0.0 witwidget 
except Exception:
  pass

In [18]:
#@title  Metrics: F1 and MCC
def f1(y_true, y_pred):
    def recall_m(label, predicted):
        tp = keras_backend.sum(keras_backend.round(keras_backend.clip(label * predicted, 0, 1)))
        positives = keras_backend.sum(keras_backend.round(keras_backend.clip(label, 0, 1)))

        return tp / (positives + keras_backend.epsilon())

    def precision_m(label, predicted):
        true_positive = keras_backend.sum(keras_backend.round(keras_backend.clip(label * predicted, 0, 1)))
        predicted_positive = keras_backend.sum(keras_backend.round(keras_backend.clip(predicted, 0, 1)))
        return true_positive / (predicted_positive + keras_backend.epsilon())

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)

    return 2 * ((precision * recall) / (precision + recall + keras_backend.epsilon()))


def mcc(y_true, y_pred):
    y_pred_pos = keras_backend.round(keras_backend.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = keras_backend.round(keras_backend.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = keras_backend.sum(y_pos * y_pred_pos)
    tn = keras_backend.sum(y_neg * y_pred_neg)

    fp = keras_backend.sum(y_neg * y_pred_pos)
    fn = keras_backend.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = keras_backend.sqrt((tp + fp) * (tn + fn) * (tn + fp) * (tp + fn))
    return numerator / (denominator + keras_backend.epsilon())

In [ ]:
#@title Load the keras models

model1 = models.load_model('first_model.h5')
model2 = models.load_model('second_model.h5')

In [21]:
# model_history = models.load_model('/content/build/')

model_history1 = models.load_model('/content/build1/', compile=False, custom_objects={'f1':f1, 'mcc':mcc}) 

In [60]:
pred = model_history1.predict(features)

array([6000.], dtype=float32)

In [ ]:
#@title Define custom prediction functions so that WIT infers using keras models

# WIT predict functions:
def custom_predict(model_inputs):
  predictions = model1.predict(model_ins)
  return predictions

In [24]:
#@title Define helper functions for dataset conversion from csv to tf.Examples

# Converts a dataframe into a list of tf.Example protos.
def df_to_examples(df, columns=None):
  examples = []
  if columns == None:
    columns = df.columns.values.tolist()
  for index, row in df.iterrows():
    example = tf.train.Example()
    for col in columns:
      if df[col].dtype is np.dtype(np.int64):
        example.features.feature[col].int64_list.value.append(int(row[col]))
      elif df[col].dtype is np.dtype(np.float64):
        example.features.feature[col].float_list.value.append(row[col])
      elif row[col] == row[col]:
        example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
    examples.append(example)
  return examples

In [54]:
#@title Read dataset and examine the algorithm

data =  pd.read_json(open("sample_data_isfahan_for_lstm.json", "r", encoding="utf8"))
# examples = df_to_examples(data)
data = data.astype(float)
data.drop(columns=['with_waiting', 'has_extra_destination', 'extra_destination_district_id', 'origin_in_even_odd'], inplace=True)
labels = np.array(data.pop('finished'))
features = np.array(data)

In [ ]:
#@title Invoke What-If Tool for the data and two models (Note that this step may take a while due to prediction speed of the toxicity model){display-mode: "form"}

Num_DataPoinst: int = 1000  #@param {type: "number"}
tool_height_in_px: int = 720  #@param {type: "number"}

# Setup the tool with the test examples and the trained classifier

config_builder = WitConfigBuilder(examples[:num_datapoints]).set_custom_predict_fn(
  custom_predict_1).set_compare_custom_predict_fn(custom_predict_2).set_custom_distance_fn(
      universal_sentence_encoder_distance)

wv = WitWidget(config_builder, height=tool_height_in_px)